In [1]:
#!/usr/bin/env python
# coding: utf-8



In [2]:
# # Prefix Consistency Gene-Level Accuracy Distribution Analysis
#
# This notebook analyzes how prefix consistency thresholds affect accuracy for individual genes within each gene type.
# It generates bar charts showing the distribution of sensitivity and false positive rates
# at different prefix consistency thresholds for each functional gene.



In [3]:
# ## Setup and Imports

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from collections import defaultdict
import os
import warnings
from IPython.display import display, Markdown
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 4)

# Define base directories (UPDATE THESE PATHS if necessary)
PREFIX_CONSISTENCY_BASE_DIR = "prefix-consistency/"
GROUND_TRUTH_BASE_DIR = "../../HPRC-assembly-benchmarking/digger-functional-annotations-all"

# List of gene types to process
GENE_TYPES_TO_PROCESS = ["IGHV", "IGLV", "IGKV", "TRAV", "TRGV", "TRDV"]



In [4]:
# ## Helper Functions for Data Loading
# 
# These functions are reused from the original code to load prefix consistency and ground truth data.



In [5]:
def load_ground_truth_data(gene_type, ground_truth_base_dir):
    """
    Load ground truth functional allele data for the given gene type.

    Args:
        gene_type (str): Gene type (e.g., "IGLV")
        ground_truth_base_dir (str): Path to the base directory containing sample ground truth folders.

    Returns:
        pd.DataFrame: DataFrame with sample_id and allele_id columns, or None if data cannot be loaded.
    """
    # For V-genes, the file prefix usually doesn't include 'V'
    file_prefix = gene_type.replace('V', '') if 'V' in gene_type else gene_type
    file_name = f"{file_prefix}_functional_alleles.txt"

    truth_data = []

    if not os.path.isdir(ground_truth_base_dir):
        print(f"❌ Error: Ground truth base directory not found: {ground_truth_base_dir}")
        return None

    # Find all sample directories
    try:
        sample_dirs = [d for d in os.listdir(ground_truth_base_dir)
                      if os.path.isdir(os.path.join(ground_truth_base_dir, d))]
    except FileNotFoundError:
        print(f"❌ Error: Could not list directories in {ground_truth_base_dir}")
        return None

    print(f"Found {len(sample_dirs)} potential sample directories in ground truth data.")
    if not sample_dirs:
        print(f"⚠️ Warning: No sample directories found in {ground_truth_base_dir}")
        return pd.DataFrame(columns=['sample_id', 'allele_id'])

    processed_samples = 0
    for sample_dir in sample_dirs:
        file_path = os.path.join(ground_truth_base_dir, sample_dir, file_name)

        # Extract sample ID from directory name
        sample_id = sample_dir.split('.')[0]

        # Check if the file exists
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                    loaded_count = 0
                    # Skip header lines starting with '#'
                    for line in lines:
                        if not line.startswith('#') and line.strip():
                            allele_id = line.strip()
                            truth_data.append({
                                'sample_id': sample_id,
                                'allele_id': allele_id
                            })
                            loaded_count += 1
                if loaded_count > 0:
                     processed_samples += 1
            except Exception as e:
                print(f"  ⚠️ Error reading {file_path}: {str(e)}")

    if not truth_data:
         print(f"⚠️ Warning: No ground truth data loaded for gene type {gene_type}.")
         return pd.DataFrame(columns=['sample_id', 'allele_id'])

    print(f"Successfully processed ground truth for {processed_samples} samples for {gene_type}.")
    # Create DataFrame from collected data
    truth_df = pd.DataFrame(truth_data)
    return truth_df



In [6]:
# ## Helper Functions for Calculating Gene-Level Metrics



In [7]:
def extract_gene_from_allele(allele_id):
    """
    Extract the gene name from an allele ID.
    
    Args:
        allele_id (str): Allele ID string (e.g., 'IGHV1-2*01')
        
    Returns:
        str: Gene name (e.g., 'IGHV1-2')
    """
    # Extract gene part (remove allele designation after *)
    if '*' in allele_id:
        return allele_id.split('*')[0]
    return allele_id

def calculate_gene_level_metrics(prefix_df):
    """
    Calculate gene-level metrics for different prefix consistency thresholds.
    
    Args:
        prefix_df (pd.DataFrame): DataFrame with prefix consistency data and ground truth status
        
    Returns:
        dict: Nested dictionary with metrics for each gene at each threshold
    """
    # Add gene column
    prefix_df['gene'] = prefix_df['allele_id'].apply(extract_gene_from_allele)
    
    # Initialize results dictionary
    gene_metrics = defaultdict(lambda: defaultdict(lambda: {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}))
    
    # Get unique genes
    genes = prefix_df['gene'].unique()
    
    # Process each threshold
    thresholds = range(6)  # 0-5
    
    for gene in genes:
        gene_df = prefix_df[prefix_df['gene'] == gene]
        
        # Skip genes with too few samples
        if len(gene_df) < 5:
            continue
            
        # Get TP and FP counts
        tp_alleles = gene_df[gene_df['status'] == 'TP']
        fp_alleles = gene_df[gene_df['status'] == 'FP']
        
        for threshold in thresholds:
            # For alleles above threshold
            tp_above = tp_alleles[tp_alleles['prefix_consistency'] >= threshold]
            fp_above = fp_alleles[fp_alleles['prefix_consistency'] >= threshold]
            
            # For alleles below threshold
            tp_below = tp_alleles[tp_alleles['prefix_consistency'] < threshold]
            fp_below = fp_alleles[fp_alleles['prefix_consistency'] < threshold]
            
            # Store counts
            gene_metrics[gene][threshold]['tp'] = len(tp_above)
            gene_metrics[gene][threshold]['fp'] = len(fp_above)
            gene_metrics[gene][threshold]['tn'] = len(fp_below)
            gene_metrics[gene][threshold]['fn'] = len(tp_below)
    
    return gene_metrics

def calculate_performance_metrics(gene_metrics):
    """
    Calculate sensitivity, false positive rate, PPV, and F-beta score for each gene at each threshold.
    
    Args:
        gene_metrics (dict): Nested dictionary with counts for each gene at each threshold
        
    Returns:
        pd.DataFrame: DataFrame with calculated metrics
    """
    results = []
    
    for gene, thresholds in gene_metrics.items():
        for threshold, counts in thresholds.items():
            # Calculate metrics
            sensitivity = counts['tp'] / (counts['tp'] + counts['fn']) if (counts['tp'] + counts['fn']) > 0 else 0
            fpr = counts['fp'] / (counts['fp'] + counts['tn']) if (counts['fp'] + counts['tn']) > 0 else 0
            # Calculate PPV (Positive Predictive Value)
            ppv = counts['tp'] / (counts['tp'] + counts['fp']) if (counts['tp'] + counts['fp']) > 0 else 0
            
            # Calculate F-beta score with beta = 0.5 (prioritizing PPV over sensitivity)
            beta = 0.5
            beta_squared = beta * beta
            f_beta = (1 + beta_squared) * (ppv * sensitivity) / (beta_squared * ppv + sensitivity) if (beta_squared * ppv + sensitivity) > 0 else 0
            
            # Store results
            results.append({
                'gene': gene,
                'threshold': threshold,
                'sensitivity': sensitivity,
                'fpr': fpr,
                'ppv': ppv,
                'f_beta': f_beta,
                'tp': counts['tp'],
                'fp': counts['fp'],
                'tn': counts['tn'],
                'fn': counts['fn'],
                'total_samples': counts['tp'] + counts['fp'] + counts['tn'] + counts['fn'],
                'passing_calls': counts['tp'] + counts['fp'],
                'passing_proportion': (counts['tp'] + counts['fp']) / (counts['tp'] + counts['fp'] + counts['tn'] + counts['fn']) 
                                     if (counts['tp'] + counts['fp'] + counts['tn'] + counts['fn']) > 0 else 0
            })
    
    return pd.DataFrame(results)



In [8]:
# ## New Functions for Finding Optimal Thresholds and Calculating Summary Metrics

def find_optimal_thresholds(metrics_df):
    """
    Find the threshold that maximizes F-beta score (beta=0.5) for each gene.
    Threshold 0 is excluded from consideration as an optimal threshold.
    
    Args:
        metrics_df (pd.DataFrame): DataFrame with gene metrics for different thresholds
        
    Returns:
        pd.DataFrame: DataFrame with optimal threshold for each gene
    """
    optimal_thresholds = []
    
    # Get all unique genes
    genes = metrics_df['gene'].unique()
    
    for gene in genes:
        # Filter data for this gene
        gene_df = metrics_df[metrics_df['gene'] == gene]
        
        # Exclude threshold 0 from consideration
        gene_df_filtered = gene_df[gene_df['threshold'] > 0]
        
        # If we have data after filtering
        if not gene_df_filtered.empty:
            # Find threshold with maximum F-beta score
            max_f_beta = gene_df_filtered['f_beta'].max()
            best_rows = gene_df_filtered[gene_df_filtered['f_beta'] == max_f_beta].sort_values('threshold', ascending=False)
            
            if not best_rows.empty:
                best_row = best_rows.iloc[0]
                
                optimal_thresholds.append({
                    'gene': gene,
                    'optimal_threshold': best_row['threshold'],
                    'f_beta': best_row['f_beta'],
                    'ppv': best_row['ppv'],
                    'sensitivity': best_row['sensitivity'],
                    'fpr': best_row['fpr'],
                    'passing_proportion': best_row['passing_proportion']
                })
    
    return pd.DataFrame(optimal_thresholds)

def calculate_gene_type_metrics(optimal_thresholds_df):
    """
    Calculate average and median metrics across all genes for a gene type.
    
    Args:
        optimal_thresholds_df (pd.DataFrame): DataFrame with optimal thresholds for each gene
        
    Returns:
        dict: Dictionary with aggregated metrics
    """
    metrics = {
        'num_genes': len(optimal_thresholds_df),
        'mean_ppv': optimal_thresholds_df['ppv'].mean(),
        'median_ppv': optimal_thresholds_df['ppv'].median(),
        'mean_sensitivity': optimal_thresholds_df['sensitivity'].mean(),
        'median_sensitivity': optimal_thresholds_df['sensitivity'].median(),
        'mean_f_beta': optimal_thresholds_df['f_beta'].mean(),
        'median_f_beta': optimal_thresholds_df['f_beta'].median(),
        'mean_passing_proportion': optimal_thresholds_df['passing_proportion'].mean(),
        'median_passing_proportion': optimal_thresholds_df['passing_proportion'].median(),
        'threshold_distribution': optimal_thresholds_df['optimal_threshold'].value_counts().to_dict()
    }
    
    return metrics

def create_summary_table(gene_types_metrics):
    """
    Create a summary table of metrics for all gene types.
    
    Args:
        gene_types_metrics (dict): Dictionary with metrics for each gene type
        
    Returns:
        pd.DataFrame: Summary table
    """
    summary_data = []
    
    for gene_type, metrics in gene_types_metrics.items():
        summary_data.append({
            'Gene Type': gene_type,
            'Number of Genes': metrics['num_genes'],
            'Mean PPV': metrics['mean_ppv'],
            'Median PPV': metrics['median_ppv'],
            'Mean Sensitivity': metrics['mean_sensitivity'],
            'Median Sensitivity': metrics['median_sensitivity'],
            'Mean F-beta (beta=0.5)': metrics['mean_f_beta'],
            'Median F-beta (beta=0.5)': metrics['median_f_beta'],
            'Mean Passing Proportion': metrics['mean_passing_proportion'],
            'Median Passing Proportion': metrics['median_passing_proportion']
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Format numeric columns
    numeric_cols = [
        'Mean PPV', 'Median PPV', 
        'Mean Sensitivity', 'Median Sensitivity',
        'Mean F-beta (beta=0.5)', 'Median F-beta (beta=0.5)',
        'Mean Passing Proportion', 'Median Passing Proportion'
    ]
    
    for col in numeric_cols:
        summary_df[col] = summary_df[col].map('{:.4f}'.format)
    
    return summary_df



In [9]:
# ## Removed per-gene plotting functionality as it's not needed
# ## We'll focus only on the summary table generation



In [10]:
# ## Main Processing Function

def process_gene_type(gene_type, collect_metrics=True):
    """
    Process data for a specific gene type.
    
    Args:
        gene_type (str): Gene type to process
        collect_metrics (bool): Whether to return metrics for summary table
        
    Returns:
        dict: Dictionary with gene type metrics (if collect_metrics=True)
    """
    display(Markdown(f"## Processing Gene Type: {gene_type}"))
    
    # Define paths
    results_dir = os.path.join(PREFIX_CONSISTENCY_BASE_DIR, f"{gene_type.lower()}/")
    all_samples_file = os.path.join(results_dir, f"all_samples_{gene_type}_prefix_consistency.csv")
    
    # Initialize return value
    gene_type_metrics = None
    
    # Check if file exists
    if not os.path.exists(all_samples_file):
        display(Markdown(f"⚠️ **Warning**: Prefix consistency file not found for {gene_type}. Please check the path."))
        display(Markdown(f"  *Expected path: `{all_samples_file}`*"))
    else:
        # Load prefix consistency data
        try:
            prefix_df = pd.read_csv(all_samples_file)
            display(Markdown(f"✅ Loaded prefix consistency data for **{gene_type}**"))
            display(Markdown(f"* Total rows: {len(prefix_df)}"))
            display(Markdown(f"* Number of samples: {prefix_df['sample_id'].nunique()}"))
            display(Markdown(f"* Number of unique alleles: {prefix_df['allele_id'].nunique()}"))
            
            # Filter to functional alleles only
            prefix_df = prefix_df[prefix_df['is_functional']].copy()
            prefix_df = prefix_df[prefix_df['in_optimal']].copy()
            display(Markdown(f"* Filtered to {len(prefix_df)} functional alleles"))
            
            # Load ground truth data
            truth_df = load_ground_truth_data(gene_type, GROUND_TRUTH_BASE_DIR)
            
            if truth_df is not None and not truth_df.empty:
                display(Markdown(f"✅ Loaded ground truth data with {len(truth_df)} entries"))
                
                # Prepare for comparison - find common samples
                prefix_samples = set(prefix_df['sample_id'].unique())
                truth_samples = set(truth_df['sample_id'].unique())
                common_samples = prefix_samples.intersection(truth_samples)
                
                display(Markdown(f"* Found {len(common_samples)} common samples for comparison"))
                
                # Filter to common samples
                prefix_df_filtered = prefix_df[prefix_df['sample_id'].isin(common_samples)].copy()
                truth_df_filtered = truth_df[truth_df['sample_id'].isin(common_samples)].copy()
                
                # Create sample-allele keys
                prefix_df_filtered['sample_allele'] = prefix_df_filtered['sample_id'] + '_' + prefix_df_filtered['allele_id']
                truth_df_filtered['sample_allele'] = truth_df_filtered['sample_id'] + '_' + truth_df_filtered['allele_id']
                
                # Find TP and FP sets
                pred_set = set(prefix_df_filtered['sample_allele'])
                truth_set = set(truth_df_filtered['sample_allele'])
                tp_set = pred_set.intersection(truth_set)
                fp_set = pred_set - truth_set
                
                # Assign TP/FP status
                prefix_df_filtered['status'] = 'Unknown'
                prefix_df_filtered.loc[prefix_df_filtered['sample_allele'].isin(tp_set), 'status'] = 'TP'
                prefix_df_filtered.loc[prefix_df_filtered['sample_allele'].isin(fp_set), 'status'] = 'FP'
                
                # Filter to known status
                known_status_df = prefix_df_filtered[prefix_df_filtered['status'].isin(['TP', 'FP'])].copy()
                
                if not known_status_df.empty:
                    display(Markdown(f"* Identified {(known_status_df['status'] == 'TP').sum()} True Positives"))
                    display(Markdown(f"* Identified {(known_status_df['status'] == 'FP').sum()} False Positives"))
                    
                    # Calculate gene-level metrics
                    display(Markdown("## Calculating Gene-Level Metrics"))
                    gene_metrics = calculate_gene_level_metrics(known_status_df)
                    
                    # Convert to DataFrame for visualization
                    metrics_df = calculate_performance_metrics(gene_metrics)
                    
                    display(Markdown(f"* Calculated metrics for {metrics_df['gene'].nunique()} genes across 6 thresholds"))
                    
                    # Display sample of metrics for inspection
                    display(Markdown("### Sample of Calculated Metrics"))
                    display(metrics_df.head(10))
                    
                    # Find optimal threshold for each gene based on F-beta score (beta=0.5)
                    display(Markdown("## Finding Optimal Thresholds Based on F-beta Score (beta=0.5)"))
                    optimal_thresholds = find_optimal_thresholds(metrics_df)
                    display(Markdown(f"* Identified optimal thresholds for {len(optimal_thresholds)} genes"))
                    display(optimal_thresholds.head())
                    
                    # Calculate gene type metrics
                    if collect_metrics:
                        gene_type_metrics = calculate_gene_type_metrics(optimal_thresholds)
                        display(Markdown("## Gene Type Metrics Summary"))
                        display(Markdown(f"* Number of genes analyzed: {gene_type_metrics['num_genes']}"))
                        display(Markdown(f"* Mean PPV: {gene_type_metrics['mean_ppv']:.4f}"))
                        display(Markdown(f"* Median PPV: {gene_type_metrics['median_ppv']:.4f}"))
                        display(Markdown(f"* Mean Sensitivity: {gene_type_metrics['mean_sensitivity']:.4f}"))
                        display(Markdown(f"* Median Sensitivity: {gene_type_metrics['median_sensitivity']:.4f}"))
                        display(Markdown(f"* Mean F-beta (beta=0.5): {gene_type_metrics['mean_f_beta']:.4f}"))
                        display(Markdown(f"* Median F-beta (beta=0.5): {gene_type_metrics['median_f_beta']:.4f}"))
                        display(Markdown(f"* Mean passing proportion: {gene_type_metrics['mean_passing_proportion']:.4f}"))
                        display(Markdown(f"* Median passing proportion: {gene_type_metrics['median_passing_proportion']:.4f}"))
                        
                        # Display threshold distribution
                        display(Markdown("* Optimal threshold distribution:"))
                        threshold_dist = pd.Series(gene_type_metrics['threshold_distribution']).sort_index()
                        display(threshold_dist)
                    
                    # Removed per-gene visualization section as it's not needed
                    
                else:
                    display(Markdown("❌ No TP or FP calls identified after comparison. Cannot proceed with analysis."))
            else:
                display(Markdown("❌ Failed to load ground truth data or no data found."))
        
        except Exception as e:
            display(Markdown(f"❌ **Error**: An error occurred during processing: {str(e)}"))
    
    return gene_type_metrics



In [11]:
# ## Main Processing Loop
# Process all gene types and generate summary table

# Initialize dictionary to store metrics for all gene types
gene_types_metrics = {}

# Process each gene type
for gene_type in GENE_TYPES_TO_PROCESS:
    gene_type_metrics = process_gene_type(gene_type)
    if gene_type_metrics:
        gene_types_metrics[gene_type] = gene_type_metrics

# Create and display summary table
if gene_types_metrics:
    display(Markdown("# Summary Table of Gene Type Metrics"))
    summary_table = create_summary_table(gene_types_metrics)
    display(summary_table)
    
    # Generate LaTeX code for the table
    latex_table = summary_table.to_latex(index=False)
    display(Markdown("## LaTeX code for the table:"))
    display(Markdown(f"```latex\n{latex_table}\n```"))
    


## Processing Gene Type: IGHV

✅ Loaded prefix consistency data for **IGHV**

* Total rows: 12693

* Number of samples: 39

* Number of unique alleles: 643

...

1    1
5    2
dtype: int64

# Summary Table of Gene Type Metrics

,Gene Type,Number of Genes,Mean PPV,Median PPV,Mean Sensitivity,Median Sensitivity,Mean F-beta (beta=0.5),Median F-beta (beta=0.5),Mean Passing Proportion,Median Passing Proportion
0,IGHV,52,0.8771,0.9310,0.3637,0.3588,0.6677,0.6948,0.3390,0.3359
1,IGLV,33,0.9633,1.0000,0.9701,0.9778,0.9635,0.9877,0.9621,0.9756
2,IGKV,37,0.8203,0.8966,0.7196,0.8043,0.7943,0.8599,0.6982,0.7692
3,TRAV,45,0.8263,1.0000,0.8770,0.9762,0.8320,0.9921,0.8786,0.9615
4,TRGV,6,0.9342,0.9762,0.9847,1.0000,0.9426,0.9785,0.9880,1.0000
5,TRDV,3,0.2403,0.0000,0.3333,0.0000,0.2545,0.0000,0.3333,0.0000


## LaTeX code for the table:

```latex
\begin{tabular}{lrllllllll}
\toprule
Gene Type & Number of Genes & Mean PPV & Median PPV & Mean Sensitivity & Median Sensitivity & Mean F-beta (beta=0.5) & Median F-beta (beta=0.5) & Mean Passing Proportion & Median Passing Proportion \\
\midrule
IGHV & 52 & 0.8771 & 0.9310 & 0.3637 & 0.3588 & 0.6677 & 0.6948 & 0.3390 & 0.3359 \\
IGLV & 33 & 0.9633 & 1.0000 & 0.9701 & 0.9778 & 0.9635 & 0.9877 & 0.9621 & 0.9756 \\
IGKV & 37 & 0.8203 & 0.8966 & 0.7196 & 0.8043 & 0.7943 & 0.8599 & 0.6982 & 0.7692 \\
TRAV & 45 & 0.8263 & 1.0000 & 0.8770 & 0.9762 & 0.8320 & 0.9921 & 0.8786 & 0.9615 \\
TRGV & 6 & 0.9342 & 0.9762 & 0.9847 & 1.0000 & 0.9426 & 0.9785 & 0.9880 & 1.0000 \\
TRDV & 3 & 0.2403 & 0.0000 & 0.3333 & 0.0000 & 0.2545 & 0.0000 & 0.3333 & 0.0000 \\
\bottomrule
\end{tabular}

```

In [12]:
# ## Simplified Summary Table
# Create a more focused table with just key metrics

if gene_types_metrics:
    display(Markdown("# Simplified Summary Table"))
    
    # Create simplified data
    simplified_data = []
    
    for gene_type, metrics in gene_types_metrics.items():
        simplified_data.append({
            'Gene Type': gene_type,
            'Mean PPV': metrics['mean_ppv'],
            'Median PPV': metrics['median_ppv'],
            'Mean Passing Proportion': metrics['mean_passing_proportion'],
            'Median Passing Proportion': metrics['median_passing_proportion']
        })
    
    # Create DataFrame
    simplified_df = pd.DataFrame(simplified_data)
    
    # Format numeric columns
    for col in ['Mean PPV', 'Median PPV', 'Mean Passing Proportion', 'Median Passing Proportion']:
        simplified_df[col] = simplified_df[col].map('{:.4f}'.format)
    
    # Display table
    display(simplified_df)
    
    # Generate LaTeX code
    latex_simplified = simplified_df.to_latex(index=False)
    display(Markdown("## LaTeX code for simplified table:"))
    display(Markdown(f"```latex\n{latex_simplified}\n```"))

# Simplified Summary Table

,Gene Type,Mean PPV,Median PPV,Mean Passing Proportion,Median Passing Proportion
0,IGHV,0.8771,0.9310,0.3390,0.3359
1,IGLV,0.9633,1.0000,0.9621,0.9756
2,IGKV,0.8203,0.8966,0.6982,0.7692
3,TRAV,0.8263,1.0000,0.8786,0.9615
4,TRGV,0.9342,0.9762,0.9880,1.0000
5,TRDV,0.2403,0.0000,0.3333,0.0000


## LaTeX code for simplified table:

```latex
\begin{tabular}{lllll}
\toprule
Gene Type & Mean PPV & Median PPV & Mean Passing Proportion & Median Passing Proportion \\
\midrule
IGHV & 0.8771 & 0.9310 & 0.3390 & 0.3359 \\
IGLV & 0.9633 & 1.0000 & 0.9621 & 0.9756 \\
IGKV & 0.8203 & 0.8966 & 0.6982 & 0.7692 \\
TRAV & 0.8263 & 1.0000 & 0.8786 & 0.9615 \\
TRGV & 0.9342 & 0.9762 & 0.9880 & 1.0000 \\
TRDV & 0.2403 & 0.0000 & 0.3333 & 0.0000 \\
\bottomrule
\end{tabular}

```